In [ ]:
# replace with your credentials
cos_credentials = {
  "apikey": "**************",
  "cos_hmac_keys": {
    "access_key_id": "**************",
    "secret_access_key": "**************"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "**************",
  "iam_apikey_name": "**************",
  "iam_role_crn": "**************",
  "iam_serviceid_crn": "**************",
  "resource_instance_id": "**************"
}
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
import random
import string
import os
import urllib
import pickle

# Return all buckets in your COS instance
def get_all_buckets(cos_client):
    response = cos_client.list_buckets()
    allbuckets = []
    for bucket in response['Buckets']:
        allbuckets.append(bucket['Name'])
    return allbuckets

# Return all the objects in a COS bucket
def get_objects_in_bucket(cos_client,bucket_name):
    return cos_client.list_objects(Bucket=bucket_name)
# Upload objects to COS bucket
def upload_file_to_bucket(cos_client,file,bucket):
    file_name = os.path.basename(file)
    print("Uploading %s to bucket: %s" % (file_name,bucket))
    cos_client.upload_file(file, bucket, file_name)
    
# Download objects from COS bucket
def download_file_from_bucket(cos_client, bucket, file_to_download, save_path, is_redownload=False):
    if not os.path.exists(save_path) or is_redownload:
        with open(save_path, 'wb') as file:
            print("Downloading %s" % file_to_download)  # "\r" allows us to overwrite the same line
            try:
                cos_client.download_fileobj(bucket, file_to_download, file)
            except:
                e = sys.exc_info()[0]
                print(e.__dict__)
                if e.response != None:
                    print("Detailed error: ", e.response)
                print('An error occured downloading %s from %s' % (file_to_download, bucket))
                os.remove(local_file)
            finally:
                file.close()

In [ ]:
# Load Python package to simplify working with COS
import boto3

In [ ]:
# Define a client for your COS instance based on the credentials
cos_client = boto3.client('s3', 
                          endpoint_url = service_endpoint, 
                          aws_access_key_id=cos_credentials["cos_hmac_keys"]["access_key_id"], 
                          aws_secret_access_key=cos_credentials["cos_hmac_keys"]["secret_access_key"])

In [ ]:
bucket = "testbucket-jkpickle"
filename = "finalized_model.sav"
directory = os.getcwd()
fname = directory + '/' + filename
download_file_from_bucket(cos_client,bucket,filename,fname)

In [ ]:
os.listdir(directory)

## WML part

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# Add WML credentials
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***********",
  "username": "*************",
  "password": "*************",
  "instance_id": "***************"
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
runtimes_meta = {
    client.runtimes.ConfigurationMetaNames.NAME: "sample", 
    client.runtimes.ConfigurationMetaNames.DESCRIPTION: "sample classifier", 
    client.runtimes.ConfigurationMetaNames.PLATFORM: { "name": "python", "version": "3.5" }
}

In [ ]:
runtime_details = client.runtimes.store(runtimes_meta)
runtime_details

In [ ]:
runtime_url = client.runtimes.get_url(runtime_details)
runtime_uid = client.runtimes.get_uid(runtime_details)
print("Runtimes URL: " + runtime_url)
print("Runtimes UID: " + runtime_uid)

In [ ]:
model_props = {client.repository.ModelMetaNames.NAME: "sample1",
               client.repository.ModelMetaNames.RUNTIME_UID: runtime_uid
              }

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
published_model = client.repository.store_model(model=loaded_model, meta_props=model_props)

In [ ]:
import json
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
created_deployment = client.deployments.create(published_model_uid, name="sample")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
print(scoring_endpoint)

In [ ]:
testExample1 = [  1.0,    90.0 ,   62.0,    12.0,    43.0,    27.2,    0.58,  24.0  ]
testExample2 = [  1.0,    60.0 ,   22.0,    12.0,    93.0,    47.2,    0.58,  24.0  ]

In [ ]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

In [ ]:
# NOTE: manually define and pass the array(s) of values to be scored in the next line
#payload_scoring = {"fields": ["","","","","","","",""], "values": [[  1.0,    90.0 ,   62.0 ,  12.0 ,   43.0,    27.2,    0.58,  24.0  ], [  1.0, 90.0, 62.0, 12.0, 43.0, 27.2, 0.78, 24.0 ]]}
payload_scoring = {"values" : [testExample1,testExample2]}

response_scoring = requests.post(scoring_endpoint, json=payload_scoring, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))